In [1]:
from google.colab import files
files.upload()

Saving hi-ud-train.conllu to hi-ud-train.conllu
Saving hi-ud-test.conllu to hi-ud-test.conllu


{'hi-ud-test.conllu': b'ID\tWORD\tTAG\r\n1\trAmAyaNa\tPROPN\r\n2\tkAla\tPROPN\r\n3\tmeM\tADP\r\n4\tBagavAna\tNOUN\r\n5\trAma\tPROPN\r\n6\tke\tADP\r\n7\tpuwra\tNOUN\r\n8\tkuSa\tPROPN\r\n9\tkI\tADP\r\n10\trAjaXAnI\tNOUN\r\n11\tkuSAvawI\tPROPN\r\n12\tko\tADP\r\n13\t483\tPROPN\r\n14\tIsA\tPROPN\r\n15\tpUrva\tPROPN\r\n16\tbuxXa\tPROPN\r\n17\tne\tADP\r\n18\tapane\tPRON\r\n19\taMwima\tADJ\r\n20\tviSrAma\tNOUN\r\n21\tke\tADP\r\n22\tlie\tADP\r\n23\tcunA\tVERB\r\n24\t.\tPUNCT\r\n\t\t\r\n1\tmalloM\tNOUN\r\n2\tkI\tADP\r\n3\trAjaXAnI\tNOUN\r\n4\thone\tVERB\r\n5\tke\tADP\r\n6\tkAraNa\tADP\r\n7\tprAcInakAla\tNOUN\r\n8\tmeM\tADP\r\n9\tisa\tDET\r\n10\tsWAna\tNOUN\r\n11\tkA\tADP\r\n12\tawyaMwa\tADJ\r\n13\tmahawva\tNOUN\r\n14\tWA\tVERB\r\n15\t.\tPUNCT\r\n\t\t\r\n1\tbOxXa\tPROPN\r\n2\tXarmAvalaMbiyoM\tNOUN\r\n3\tke\tADP\r\n4\tanusAra\tADP\r\n5\tluMbanI\tPROPN\r\n6\t","\tPUNCT\r\n7\tboXagayA\tPROPN\r\n8\tOra\tCCONJ\r\n9\tsAranAWa\tPROPN\r\n10\tke\tADP\r\n11\tsAWa\tADP\r\n12\thI\tPART\r\n13\tisa\tDET\r\n14\

In [150]:
!pip install sklearn-crfsuite

from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [151]:
test_file = open('hi-ud-test.conllu',encoding = "utf-8")
train_file = open('hi-ud-train.conllu',encoding = "utf-8")
test_data = list(test_file)
train_data = list(train_file)

In [152]:
temp = []

#remove white spaces around data
for i in range(len(train_data)):
  temp.append(train_data[i].strip())

train_data = temp

train_sent = []
temp = []

for i in range(1,len(train_data)):
  sent = train_data[i]
  if sent == ",,":
    train_sent.append(temp)
    temp = []
  else :
    sent = sent.encode('ascii', 'ignore')
    sent = sent.decode('ascii')
    sent = sent.split(',')
    temp.append(sent)


temp = []

for i in range(len(test_data)):
  temp.append(test_data[i].strip())

test_data = temp

test_sent = []
temp = []

for i in range(1,len(test_data)):
  sent = test_data[i]
  if sent == '' :
    test_sent.append(temp)
    temp = []
  else :
    sent = sent.encode('ascii', 'ignore')
    sent = sent.decode('ascii')
    sent = sent.split('\t')
    temp.append(sent)


## Features Chosen:
**Word**           - The current word

**word.Lower()**   - The word in lowercase  

**Suf_2**       - word[-2:] , last two characters

**Suf_3**       - word[-3:] , last three characters

**Pre_2**       - word[:2] , first two characters

**Pre_3**       - word[:3] , first three characters

**word.istitle()** - True if first character is in UpperCase  

**word.isupper()** - True if all characters of the string are UpperCase  

**word.isdigit()** - True if word is a number   

**BOS**               - If word is the Beginning of the Sentence  

**-1:word.Lower()**   - Previous word in LowerCase   

**-1:word.isTitle()** - True if first character of the Previous word is in UpperCase  

**-1:word.isUpper()** - True if all characters of the Previous word are UpperCase  

**EOS**               - If the word is the End of the Sentence  

**+1:word.Lower()**   - next word in LowerCase  

**+1:word.isTitle()** - True if first character of the Next word is in UpperCase  

**+1:word.isUpper()** - True if all characters of the Next word are UpperCase  

In [153]:
def word2features(sent, i):
    
    word = sent[i][1]
    postag = sent[i][2]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'word.isupper()': word.isupper(), 
        'word.islower()': word.islower(), 
        'word.istitle()': word.istitle(), 
        'word.isdigit()': word.isdigit(), 
        'word.isalpha_numeric()': word.isalnum() 
    }
    if i > 0:
        word1 = sent[i-1][1]
        postag1 = sent[i-1][2]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper()
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][1]
        postag1 = sent[i+1][2]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper()
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2pos(sent):
    return [x[2] for x in sent]


x_train = []
x_test = []
y_test = []
y_train = []

for i in range(len(train_sent)):
    sent = train_sent[i]
    x_train.append(sent2features(sent))

for i in range(len(train_sent)):
    sent = train_sent[i]
    y_train.append(sent2pos(sent))

for i in range(len(test_sent)):
    sent = test_sent[i]
    x_test.append(sent2features(sent))

for i in range(len(test_sent)):
    sent = test_sent[i]
    y_test.append(sent2pos(sent))


In [154]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(x_train, y_train)

labels = list(crf.classes_)
print(labels)

y_pred_test = crf.predict(x_test)
y_pred_train = crf.predict(x_train)

['DET', 'PROPN', 'ADP', 'ADV', 'ADJ', 'NOUN', 'NUM', 'AUX', 'PUNCT', 'PRON', 'VERB', 'CCONJ', 'PART', 'COMMA', 'SCONJ', 'X']


In [158]:
print("Accuracy on train data : ", metrics.flat_accuracy_score(y_train,y_pred_train))
print("Accuracy on test data : ", metrics.flat_accuracy_score(y_test,y_pred_test))

Accuracy on train data :  0.9968412740194789
Accuracy on test data :  0.8580332409972299


In [159]:

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top 10 most common transitions:")
print_transitions(Counter(crf.transition_features_).most_common(10))

Top 10 most common transitions:
VERB   -> AUX     3.940531
PROPN  -> PROPN   2.642134
ADJ    -> NOUN    2.347299
AUX    -> AUX     2.169317
PROPN  -> ADP     2.164068
PROPN  -> COMMA   2.076191
NUM    -> NOUN    2.029694
PRON   -> ADP     1.878091
DET    -> NOUN    1.798439
NOUN   -> ADP     1.792727


In [160]:
print("\nTop 10 least common transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-10:])


Top 10 least common transitions:
NUM    -> PRON    -1.141196
AUX    -> ADJ     -1.182282
ADV    -> AUX     -1.202015
PROPN  -> DET     -1.285714
PROPN  -> AUX     -1.309683
COMMA  -> ADP     -1.624423
CCONJ  -> AUX     -1.642771
DET    -> ADP     -1.977607
ADJ    -> ADP     -2.079606
ADJ    -> PRON    -2.112368


In [161]:
sorted_labels = sorted(labels, key=lambda name: (name, name[0]))

print("Training data results : ")
print(metrics.flat_classification_report(y_train, y_pred_train, labels=sorted_labels, digits=3))


Training data results : 
              precision    recall  f1-score   support

         ADJ      1.000     1.000     1.000       569
         ADP      0.998     0.999     0.998      1384
         ADV      0.991     0.982     0.986       110
         AUX      0.985     1.000     0.993       728
       CCONJ      0.993     1.000     0.997       150
       COMMA      0.991     1.000     0.996       114
         DET      0.996     0.991     0.993       230
        NOUN      0.998     0.999     0.999      1596
         NUM      1.000     1.000     1.000       152
        PART      1.000     1.000     1.000       163
        PRON      0.995     0.995     0.995       430
       PROPN      1.000     0.996     0.998       707
       PUNCT      1.000     1.000     1.000       563
       SCONJ      0.984     1.000     0.992        61
        VERB      1.000     0.981     0.991       639
           X      1.000     1.000     1.000         2

    accuracy                          0.997      7598
 

In [162]:
print("\nTest data results : ")
print(metrics.flat_classification_report(y_test, y_pred_test, labels=sorted_labels, digits=3))


Test data results : 
              precision    recall  f1-score   support

         ADJ      0.696     0.755     0.724        94
         ADP      0.967     0.970     0.969       303
         ADV      0.667     0.476     0.556        21
         AUX      0.936     0.949     0.942       138
       CCONJ      1.000     1.000     1.000        25
       COMMA      0.000     0.000     0.000         0
         DET      0.889     0.889     0.889        36
        NOUN      0.772     0.889     0.826       324
         NUM      1.000     0.880     0.936        25
        PART      1.000     0.939     0.969        33
        PRON      0.885     0.831     0.857        65
       PROPN      0.635     0.507     0.564       144
       PUNCT      1.000     0.896     0.945       134
       SCONJ      0.750     1.000     0.857         3
        VERB      0.885     0.859     0.872        99
           X      0.000     0.000     0.000         0

   micro avg      0.858     0.858     0.858      1444
   m

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
